# Лабораторна робота 1 з Симетричної Криптографії
Команда: Бондар, Дигас
Група: ФІ-03

## Підготовка даних
1. В якості середньостатистичного тексту російською мовою ми взяли дописи з телеграм-каналу терориста і військового злочинця ігоря гіркіна
2. Повний вхідний текст можна знайти у відповідному файлі
3. Обробка тексту і підготовка до аналізу відбувається у розділі "Text reading and preprocessing"

## Програмна частина

### Text reading and preprocessing

In [2]:
filename = "girkin_crying.txt"

In [3]:
def get_text(_filename):
    f = open(_filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return text

def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё':
        return 'е'
    else:
        return ' '
    
def preprocess_text(_text):
    _text = get_text(filename)
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)

    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

In [4]:
text = preprocess_text(get_text(filename))

### Text processing (singular char count and bigram count)

In [5]:
def count_chars(_text):
    c_count = {}
    for c in _text:
        if c not in c_count:
            c_count[c] = 1
        else:
            c_count[c] = c_count[c] + 1 

    return dict(sorted(c_count.items()))

# Bigrams with intersection (ex: [1, 2], [2, 3], [3, 4])
def count_bigrams_w_i(_text):
    b_count = {}
    prev_char = _text[0]
    for c in _text[1:]:
        bg = prev_char + c
        prev_char = c
        if bg not in b_count:
            b_count[bg] = 1
        else:
            b_count[bg] = b_count[bg] + 1 

    return dict(sorted(b_count.items()))

# Bigrams without intersection (ex: [1, 2], [3, 4])
def count_bigrams_wo_i(_text):
    b_count = {}
    i = 1
    while i < len(_text):
        bg = _text[i - 1] + _text[i]
        if bg not in b_count:
            b_count[bg] = 1
        else:
            b_count[bg] = b_count[bg] + 1 
        i = i + 2

    return dict(sorted(b_count.items()))

In [6]:
chars_freq_wspaces = count_chars(text)
chars_freq_wospaces = chars_freq_wspaces.copy()
del chars_freq_wospaces[' ']

bigrams_freq_w_intersect = count_bigrams_w_i(text)
bigrams_freq_wo_intersect = count_bigrams_wo_i(text)

### Show symbol frequencies

In [7]:
for k, v in chars_freq_wspaces.items():
    print(f"{k} : {v}")

  : 86793
а : 37904
б : 8174
в : 22994
г : 7676
д : 14226
е : 43426
ж : 4720
з : 7408
и : 37250
й : 6380
к : 16426
л : 18174
м : 15420
н : 36030
о : 56742
п : 15740
р : 24918
с : 26566
т : 31194
у : 12712
ф : 1612
х : 5356
ц : 2422
ч : 6828
ш : 3240
щ : 1846
ъ : 164
ы : 8952
ь : 7650
э : 1236
ю : 3406
я : 9048


### Show bigram frequencies

In [54]:
alph = list(chars_freq_wspaces.keys())
print("   ", end='|')
for l_c in alph:
    print(f" '{l_c}' ", end='|')
for l_c in alph:
    print(f"\n'{l_c}'", end='|')
    for r_c in alph:
        k = l_c + r_c
        if k in bigrams_freq_w_intersect:
            print(f"{bigrams_freq_w_intersect[k]:5d}", end='|')
        else:
            print(f"{0:5d}", end='|')

   | ' ' | 'а' | 'б' | 'в' | 'г' | 'д' | 'е' | 'ж' | 'з' | 'и' | 'й' | 'к' | 'л' | 'м' | 'н' | 'о' | 'п' | 'р' | 'с' | 'т' | 'у' | 'ф' | 'х' | 'ц' | 'ч' | 'ш' | 'щ' | 'ъ' | 'ы' | 'ь' | 'э' | 'ю' | 'я' |
' '|    0| 1790| 3314| 9052| 1754| 3626| 1146|  514| 2046| 5760|   54| 4112| 1376| 3208| 9268| 5460|10444| 3268| 7726| 4094| 2800|  728|  749|  320| 2080|  382|    6|    0|    0|    0| 1104|   72|  540|
'а'| 7586|   10|  488| 1636|  560| 1036|  954|  856| 1386|  298|  468| 2572| 2688| 1642| 3256|   54|  594| 1914| 2356| 2938|   48|   48|  672|  804|  690|  664|  122|    0|    0|    0|    4|  762|  798|
'б'|  152|  500|    2|   12|    0|   20| 1048|   36|    6|  566|    0|   32|  494|   74|  264| 1704|   20|  582|  192|    6|  666|    0|  106|    0|    0|    4|  294|  140| 1070|   32|    0|   16|  136|
'в'| 4476| 2924|    8|   50|    8|   94| 2190|    2|  216| 1430|    0|  372|  798|  108| 1220| 4218|  168|  598| 1558|  196|  304|    0|    2|   28|   66|  302|   12|    0| 1386|   64|    

### Calculate $H_1$ and $H_2$

In [9]:
import math

char_amount = sum(chars_freq_wspaces.values())
t1 = [chars_freq_wspaces[k] / char_amount for k in chars_freq_wspaces.keys()]
H1 = -sum(a * math.log2(a) for a in t1)

bg_amount = sum(bigrams_freq_w_intersect.values())
t2 = [bigrams_freq_w_intersect[k] / bg_amount for k in bigrams_freq_w_intersect.keys()]
H2 = -sum(a * math.log2(a) for a in t2) / 2
print(f"H1 = {H1}")
print(f"H2 = {H2}")

H1 = 4.385129362944809
H2 = 3.9881215496418245


## Обчислення $H^{(10)}$, $H^{(20)}$, $H^{(30)}$ за допомогою Cool Pink Program

<img src="images/H10.png">
<img src="images/H20.png">
<img src="images/H30.png">

Отримані результати:
$
\begin{gather}
1.2595 \leq H^{(10)} \leq 2.0221\\
1.3234 \leq H^{(20)} \leq 2.0679\\
1.3960 \leq H^{(30)} \leq 2.0556
\end{gather}
$

Обчислимо надлишковість:

Так як $H_0 = \log_ 2(32) = 5$, то за формулою надлишковості $R = 1 - \frac{H_{\infty}}{H_0}$:

Розглядаємо $H^{(i)}$ як наближення $H_{\infty}$.
$
\begin{gather}
\begin{array}{cc}
    H^{(10)}:&\; 0.404 \leq R \leq 0.748\\
    H^{(20)}:&\; 0.464 \leq R \leq 0.735\\
    H^{(30)}:&\; 0.588 \leq R \leq 0.721
\end{array}
\end{gather}
$